In [2]:
import csv
import pandas as pd
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point


In [3]:

# Load the existing mine locations from the csv file
mines_df = pd.read_csv('/Users/kd6801/Desktop/African-mining/all_mining_location.csv')

mines_df = mines_df.rename(columns={
    'longitude': 'Longitude',
    'latitude': 'Latitude'
})

mines_gdf = gpd.GeoDataFrame(
    mines_df, 
    geometry=gpd.points_from_xy(mines_df.Longitude, mines_df.Latitude)
)


# Load the NDVI loss coordinates
ndvi_loss_data = pd.read_csv('/Users/kd6801/Desktop/African-mining/combined_mining.csv')
ndvi_loss_df = pd.DataFrame(ndvi_loss_data)
ndvi_loss_gdf = gpd.GeoDataFrame(
    ndvi_loss_df, 
    geometry=gpd.points_from_xy(ndvi_loss_df.Longitude, ndvi_loss_df.Latitude)
)


In [4]:
#Set the coordinate reference system (CRS) to WGS 84 (EPSG:4326)
mines_gdf.set_crs(epsg=4326, inplace=True)
ndvi_loss_gdf.set_crs(epsg=4326, inplace=True)

,Longitude,Latitude,geometry
0,16.655888,3.910950,POINT (16.65589 3.91095)
1,16.653373,3.912837,POINT (16.65337 3.91284)
2,16.657146,3.912837,POINT (16.65715 3.91284)
3,16.652744,3.913466,POINT (16.65274 3.91347)
4,16.653373,3.913466,POINT (16.65337 3.91347)
...,...,...,...
4708,17.675835,6.003037,POINT (17.67584 6.00304)
4709,17.682752,6.004923,POINT (17.68275 6.00492)
4710,17.676464,6.011840,POINT (17.67646 6.01184)
4711,17.673320,6.012469,POINT (17.67332 6.01247)


In [5]:

# Convert the CRS to a projected system to measure distances in meters
mines_gdf = mines_gdf.to_crs(epsg=32633)  # UTM Zone 33N (modify based on your area)
ndvi_loss_gdf = ndvi_loss_gdf.to_crs(epsg=32633)  # UTM Zone 33N (modify based on your area)

# Perform a spatial join to find points within 1 km radius
radius = 10000 # 10 km

# Buffer the mine locations by 1 km
mines_buffered = mines_gdf.buffer(radius)

# Check if NDVI loss points are within any of the buffered mine locations
ndvi_loss_gdf['potential_mining'] = ndvi_loss_gdf.apply(
    lambda row: mines_buffered.contains(row.geometry).any(), axis=1
).astype(int)

# Convert the result to a DataFrame and display it
result_df = ndvi_loss_gdf[['Longitude', 'Latitude', 'potential_mining']]
result_df.head()

,Longitude,Latitude,potential_mining
0,16.655888,3.910950,0
1,16.653373,3.912837,0
2,16.657146,3.912837,0
3,16.652744,3.913466,0
4,16.653373,3.913466,0


In [6]:
result_df

,Longitude,Latitude,potential_mining
0,16.655888,3.910950,0
1,16.653373,3.912837,0
2,16.657146,3.912837,0
3,16.652744,3.913466,0
4,16.653373,3.913466,0
...,...,...,...
4708,17.675835,6.003037,0
4709,17.682752,6.004923,0
4710,17.676464,6.011840,0
4711,17.673320,6.012469,0


In [7]:
print(len(result_df[result_df['potential_mining'] == 1]))

910


In [8]:
# Create a base map
m = folium.Map(location=[3.910950, 16.655888], zoom_start=6)

# Add existing mine locations to the map with blue circles
for idx, row in mines_gdf.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        popup=f"Mine Location\nLongitude: {row['Longitude']}\nLatitude: {row['Latitude']}",
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.3  # More transparent
    ).add_to(m)

# Add NDVI loss locations to the map with red circles
for idx, row in ndvi_loss_gdf.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        popup=f"NDVI Loss Location\nLongitude: {row['Longitude']}\nLatitude: {row['Latitude']}",
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7  # Slightly less transparent
    ).add_to(m)

# Add a legend to the map
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 150px; height: 90px; 
                 background-color: white; z-index:9999; font-size:14px;
                 border: 2px solid grey;
                 ">
     &nbsp;<b>Legend</b><br>
     &nbsp;<i class="fa fa-circle" style="color:blue"></i>&nbsp; Mine Locations<br>
     &nbsp;<i class="fa fa-circle" style="color:red"></i>&nbsp; NDVI Loss Locations
     </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
m.save('mining_and_ndvi_loss_map.html')

# Display the map
#m


In [9]:
mines_gdf_labeled = result_df.copy()
print(f"After matching with radius = {radius} meters, we get {len(mines_gdf_labeled[mines_gdf_labeled['potential_mining'] == 1])} potential mining locations.")
print("The number of non-mining locations is: ", len(mines_gdf_labeled[mines_gdf_labeled['potential_mining'] == 0]))

# We label the existing mines as potential mining locations
existing_mines_labeled = mines_gdf.copy()
existing_mines_labeled['potential_mining'] = 1
existing_mines_labeled_clean = existing_mines_labeled[['Longitude', 'Latitude', 'potential_mining']]
existing_mines_labeled_clean

After matching with radius = 10000 meters, we get 910 potential mining locations.
The number of non-mining locations is:  3803


,Longitude,Latitude,potential_mining
0,17.838062,3.830327,1
1,17.467914,4.794698,1
2,17.837117,3.833920,1
3,17.464800,4.796248,1
4,17.502983,4.760425,1
...,...,...,...
909,16.125267,3.561943,1
910,16.236954,3.403430,1
911,16.245850,3.401320,1
912,16.248991,3.257713,1


In [10]:
combined = pd.merge(existing_mines_labeled_clean,mines_gdf_labeled, on=['Longitude', 'Latitude'], how='outer')
# Combine the potential_mining columns into a single column
combined['potential_mining'] = combined['potential_mining_x'].combine_first(combined['potential_mining_y'])

# Drop the original columns with the suffixes
combined = combined.drop(columns=['potential_mining_x', 'potential_mining_y'])

# Print the combined DataFrame
print(combined)
print(f"After combining, we have {len(combined[combined['potential_mining'] == 1])} potential mining locations.")

      Longitude  Latitude  potential_mining
0     17.838062  3.830327               1.0
1     17.838062  3.830327               1.0
2     17.838062  3.830327               1.0
3     17.467914  4.794698               1.0
4     17.467914  4.794698               1.0
...         ...       ...               ...
5622  17.675835  6.003037               0.0
5623  17.682752  6.004923               0.0
5624  17.676464  6.011840               0.0
5625  17.673320  6.012469               0.0
5626  17.673949  6.012469               0.0

[5627 rows x 3 columns]
After combining, we have 1824 potential mining locations.


In [11]:
# combined['Match'] = combined['_merge'].apply(lambda x: 1 if x == 'both' else 0)
#combined.to_csv("marked_match.csv")

## Data Cleaning (local): add distance to the River/Village/Road/Protected areas

In [27]:
# Load Road data:
# Roads data
Road_path = '/Users/kd6801/Desktop/Mining-Project/CAF-Roads/CAR_DATA_140731_Roads.shp'
gdf_road = gpd.read_file(Road_path)
# Villages data
Village_path = '/Users/kd6801/Desktop/Mining-Project/CAF-Villages/caf_settlements_sigcaf.shp'
gdf_village = gpd.read_file(Village_path)
# CAF waterway lines
Water_path = '/Users/kd6801/Desktop/Mining-Project/CAF-Waterway-lines/hotosm_caf_waterways_lines_shp.shp'
gdf_water = gpd.read_file(Water_path)


In [36]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point

# Load your data into GeoDataFrames
# Assuming `mines_gdf`, `ndvi_loss_gdf`, and `gdf_village` are already loaded

# Create a base map
m = folium.Map(location=[3.910950, 16.655888], zoom_start=6)

# Add existing mine locations to the map with blue circles
for idx, row in mines_gdf.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        popup=f"Mine Location\nLongitude: {row['Longitude']}\nLatitude: {row['Latitude']}",
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.3  # More transparent
    ).add_to(m)

# Add NDVI loss locations to the map with red circles
for idx, row in ndvi_loss_gdf.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        popup=f"NDVI Loss Location\nLongitude: {row['Longitude']}\nLatitude: {row['Latitude']}",
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7  # Slightly less transparent
    ).add_to(m)

# Add village locations to the map with green circles
for idx, row in gdf_village.iterrows():
    folium.CircleMarker(
        location=[row['Y_Latitude'], row['X_longitud']],
        radius=5,
        popup=f"Village\nName: {row['PName1']}\nLongitude: {row['X_longitud']}\nLatitude: {row['Y_Latitude']}",
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.5  # More transparent
    ).add_to(m)

# Add a legend to the map
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 200px; height: 120px; 
                 background-color: white; z-index:9999; font-size:14px;
                 border: 2px solid grey;
                 ">
     &nbsp;<b>Legend</b><br>
     &nbsp;<i class="fa fa-circle" style="color:blue"></i>&nbsp; Mine Locations<br>
     &nbsp;<i class="fa fa-circle" style="color:red"></i>&nbsp; NDVI Loss Locations<br>
     &nbsp;<i class="fa fa-circle" style="color:green"></i>&nbsp; Village Locations
     </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
m.save('villages-mining-ndvi-loss.html')


In [56]:
combined_data = pd.DataFrame(combined)

combined_gdf = gpd.GeoDataFrame(
    combined_data, 
    geometry=gpd.points_from_xy(combined.Longitude, combined.Latitude),
    crs="EPSG:32633"
)

# Set the initial CRS if not set (using WGS 84 as an example)
if gdf_road.crs is None:
    gdf_road.set_crs(epsg=32633, inplace=True)
if gdf_village.crs is None:
    gdf_village.set_crs(epsg=32633, inplace=True)
if gdf_water.crs is None:
    gdf_water.set_crs(epsg=32633, inplace=True)

# Transform all GeoDataFrames to the desired CRS (EPSG:3857)
combined_gdf = combined_gdf.to_crs(epsg=32633)
gdf_road = gdf_road.to_crs(epsg=32633)
gdf_village = gdf_village.to_crs(epsg=32633)
gdf_water = gdf_water.to_crs(epsg=32633)


print(combined_gdf.crs)
print(gdf_road.crs)
print(gdf_village.crs)
print(gdf_water.crs)

EPSG:32633
EPSG:32633
EPSG:32633
EPSG:32633


In [60]:
# I only want to keep the locations of the villages
villages_gdf_loc = gdf_village[['geometry']]
villages_gdf_loc


# Function to calculate the closest distance
def calculate_nearest_distance(gdf1, gdf2):
    distances = []
    for geom in gdf1.geometry:
        nearest_geom = gdf2.geometry[gdf2.distance(geom).idxmin()]
        nearest_dist = geom.distance(nearest_geom)
        distances.append(nearest_dist)
    return distances

# Calculate the closest distance from each combined location to the nearest village
combined_gdf['distance_to_village'] = calculate_nearest_distance(combined_gdf, villages_gdf_loc)

# Display the updated DataFrame
print(combined_gdf[['Longitude', 'Latitude', 'distance_to_village']])

      Longitude  Latitude  distance_to_village
0     17.838062  3.830327             0.014054
1     17.838062  3.830327             0.014054
2     17.838062  3.830327             0.014054
3     17.467914  4.794698             0.065802
4     17.467914  4.794698             0.065802
...         ...       ...                  ...
5622  17.675835  6.003037             0.193725
5623  17.682752  6.004923             0.200703
5624  17.676464  6.011840             0.199617
5625  17.673320  6.012469             0.197438
5626  17.673949  6.012469             0.197951

[5627 rows x 3 columns]


In [69]:
# Calculate the closest distance from each combined location to the nearest waterway and road
combined_gdf['distance_to_waterway'] = calculate_nearest_distance(combined_gdf, gdf_water)
combined_gdf['distance_to_road'] = calculate_nearest_distance(combined_gdf, gdf_road)


# Display the updated DataFrame
print(combined_gdf[['Longitude', 'Latitude', 'distance_to_village_m', 'distance_to_waterway', 'distance_to_road']])

# Optionally, save the updated GeoDataFrame to a file
combined_gdf.to_csv('combined_central_africa.csv', index=False)


In [73]:
combined_gdf_path = '/Users/kd6801/Desktop/Mining-Project/CAF-Roads/combined_central_africa.csv'
combined_gdf_complete = pd.read_csv(combined_gdf_path)

# In meters
combined_gdf_complete['distance_to_waterway'] = combined_gdf_complete['distance_to_waterway'] * 1000
combined_gdf_complete['distance_to_road'] = combined_gdf_complete['distance_to_road'] * 1000
combined_gdf_complete['distance_to_village'] = combined_gdf_complete['distance_to_village'] * 1000


In [74]:

combined_gdf_complete

,Unnamed: 0,Longitude,Latitude,potential_mining,geometry,distance_to_village,distance_to_waterway,distance_to_road
0,0,17.838062,3.830327,1.0,POINT (17.83806167 3.83032667),13.623132,23.943586,28.040273
1,1,17.838062,3.830327,1.0,POINT (17.83806167 3.83032667),13.623132,23.943586,28.040273
2,2,17.838062,3.830327,1.0,POINT (17.83806167 3.83032667),13.623132,23.943586,28.040273
3,3,17.467914,4.794698,1.0,POINT (17.467914 4.794698),63.974506,119.191732,5.957668
4,4,17.467914,4.794698,1.0,POINT (17.467914 4.794698),63.974506,119.191732,5.957668
...,...,...,...,...,...,...,...,...
5622,5622,17.675835,6.003037,0.0,POINT (17.67583543527038 6.003036801892908),187.576964,145.413453,184.839059
5623,5623,17.682752,6.004923,0.0,POINT (17.6827524629581 6.004923263989559),194.317867,142.633832,191.547264
5624,5624,17.676464,6.011840,0.0,POINT (17.67646425596926 6.011840291677279),193.325401,151.941107,190.677585
5625,5625,17.673320,6.012469,0.0,POINT (17.67332015247485 6.012469112376163),191.231385,154.353304,188.509261


In [75]:
without_existing_combined = pd.read_csv('/Users/kd6801/Desktop/Mining-Project/CAF-Roads/without_existing_combined.csv')

## Sensitivity Analysis: Using the Central Africa Data

In [76]:
df = pd.DataFrame(combined_gdf_complete)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Features and target variable
X = df[['Longitude', 'Latitude', 'distance_to_village', 'distance_to_waterway', 'distance_to_road']]
y = df['potential_mining']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))

# Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
print("Gradient Boosting Accuracy:", accuracy_score(y_test, y_pred_gb))
print("Gradient Boosting Confusion Matrix:\n", confusion_matrix(y_test, y_pred_gb))
print("Gradient Boosting Classification Report:\n", classification_report(y_test, y_pred_gb))


Random Forest Accuracy: 0.9946714031971581
Random Forest Confusion Matrix:
 [[741   4]
 [  2 379]]
Random Forest Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.99      1.00       745
         1.0       0.99      0.99      0.99       381

    accuracy                           0.99      1126
   macro avg       0.99      0.99      0.99      1126
weighted avg       0.99      0.99      0.99      1126

Gradient Boosting Accuracy: 0.9786856127886323
Gradient Boosting Confusion Matrix:
 [[727  18]
 [  6 375]]
Gradient Boosting Classification Report:
               precision    recall  f1-score   support

         0.0       0.99      0.98      0.98       745
         1.0       0.95      0.98      0.97       381

    accuracy                           0.98      1126
   macro avg       0.97      0.98      0.98      1126
weighted avg       0.98      0.98      0.98      1126



In [79]:
df_without = pd.DataFrame(without_existing_combined)


from sklearn.model_selection import train_test_split

# Features and target variable
X = df_without[['Longitude', 'Latitude', 'distance_to_village', 'distance_to_waterway', 'distance_to_road']]
y = df_without['potential_mining']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))

# Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
print("Gradient Boosting Accuracy:", accuracy_score(y_test, y_pred_gb))
print("Gradient Boosting Confusion Matrix:\n", confusion_matrix(y_test, y_pred_gb))
print("Gradient Boosting Classification Report:\n", classification_report(y_test, y_pred_gb))


Random Forest Accuracy: 0.9946977730646872
Random Forest Confusion Matrix:
 [[761   3]
 [  2 177]]
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       764
           1       0.98      0.99      0.99       179

    accuracy                           0.99       943
   macro avg       0.99      0.99      0.99       943
weighted avg       0.99      0.99      0.99       943

Gradient Boosting Accuracy: 0.9830328738069989
Gradient Boosting Confusion Matrix:
 [[754  10]
 [  6 173]]
Gradient Boosting Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       764
           1       0.95      0.97      0.96       179

    accuracy                           0.98       943
   macro avg       0.97      0.98      0.97       943
weighted avg       0.98      0.98      0.98       943



In [86]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
import numpy as np
import joblib
# Prepare the data for modeling
X = df[['distance_to_village', 'distance_to_waterway', 'distance_to_road']]
y = df['potential_mining']

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split the resampled data
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Random Forest with class weights
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced')
rf_model.fit(X_train_resampled, y_train_resampled)
y_pred_rf = rf_model.predict(X_test_resampled)

# Evaluation Metrics
print("Random Forest Accuracy:", accuracy_score(y_test_resampled, y_pred_rf))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_test_resampled, y_pred_rf))
print("Random Forest Classification Report:\n", classification_report(y_test_resampled, y_pred_rf))

# Calculate different metrics
precision = precision_score(y_test_resampled, y_pred_rf)
recall = recall_score(y_test_resampled, y_pred_rf)
f1 = f1_score(y_test_resampled, y_pred_rf)
roc_auc = roc_auc_score(y_test_resampled, y_pred_rf)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("AUC-ROC:", roc_auc)

# Cross-validation for Random Forest
rf_cv_scores = cross_val_score(rf_model, X_resampled, y_resampled, cv=5)
print("Random Forest Cross-Validation Scores:", rf_cv_scores)
print("Random Forest Cross-Validation Mean Score:", np.mean(rf_cv_scores))

# Feature importance for Random Forest
importances = rf_model.feature_importances_
feature_names = X.columns
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print("Feature Importances:\n", feature_importances)

# Save the Random Forest model
joblib.dump(rf_model, 'random_forest_model.pkl')


Random Forest Accuracy: 0.9034165571616294
Random Forest Confusion Matrix:
 [[700  62]
 [ 85 675]]
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.92      0.90       762
           1       0.92      0.89      0.90       760

    accuracy                           0.90      1522
   macro avg       0.90      0.90      0.90      1522
weighted avg       0.90      0.90      0.90      1522

Precision: 0.9158751696065129
Recall: 0.8881578947368421
F1-score: 0.9018036072144289
AUC-ROC: 0.9033965326702582
Random Forest Cross-Validation Scores: [0.71681997 0.93293886 0.95463511 0.82708744 0.7251808 ]
Random Forest Cross-Validation Mean Score: 0.8313324365583539
Feature Importances:
                 Feature  Importance
1  distance_to_waterway    0.345939
0   distance_to_village    0.329964
2      distance_to_road    0.324097


['random_forest_model.pkl']

In [87]:
df.columns

Index(['Unnamed: 0', 'Longitude', 'Latitude', 'geometry', 'potential_mining',
       'distance_to_village', 'distance_to_waterway', 'distance_to_road'],
      dtype='object')